In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from subprocess import CREATE_NO_WINDOW

import requests
from bs4 import BeautifulSoup

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import requests

import subprocess as sp # 쉘 명령어를 실행하기 위한 모듈
import re # 정규표현식 사용
import json

In [17]:
class CurrentLocation:
    def __init__(self):
        self.lat: float = 0
        self.long: float = 0
        self.radius: int = 0
        self.address: str = ""

        # 정확도 조절 - 높은 값으로 설정할수록 속도가 빨라지나 정확도가 떨어짐
        accuracy = 1
        commd = 'add-type -assemblyname system.device; '\
                '$loc = new-object system.device.location.geocoordinatewatcher;'\
                '$loc.start(); '\
                'while(($loc.status -ne "Ready") -and ($loc.permission -ne "Denied")) '\
                '{start-sleep -milliseconds 100}; '\
                '$acc = %d; '\
                'while($loc.position.location.horizontalaccuracy -gt $acc) '\
                '{start-sleep -milliseconds 100; $acc = [math]::Round($acc*1.5)}; '\
                '$loc.position.location.latitude; '\
                '$loc.position.location.longitude; '\
                '$loc.position.location.horizontalaccuracy; '\
                '$loc.stop()' %(accuracy)

        pshellcomm = ['powershell', commd]

        # 쉘 명령어 실행 및 결과값 저장
        p = sp.Popen(pshellcomm, stdin = sp.PIPE, stdout = sp.PIPE, stderr = sp.STDOUT, text=True)
        (out, err) = p.communicate()

        # 전체 출력 문자열에서 개행문자를 기준으로 분리 
        out = re.split('\n', out)

        self.lat = float(out[0])
        self.long = float(out[1])
        self.radius = int(out[2])

        # 위도, 경도를 주소 문자열로 변환
        url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=self.long,latitude=self.lat)
        headers = {"Authorization": "KakaoAK " + "82e74383a81e76a453aed30fac79cef2"}
        result = json.loads(str(requests.get(url, headers=headers).text))
        match_first = result['documents'][0]['address_name']

        self.address = str(match_first)
        
        
    def restaurant_list(self):        
        options = Options()
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")
        service = Service(ChromeDriverManager().install())

        url = f'https://map.kakao.com/?q={self.address}+맛집'
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        
        #식당 정보 얻어오는 코드
        restaurant = driver.find_elements(By.CLASS_NAME, 'link_name')
        star = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[4]/span[1]/em')
        address = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[5]/div[2]/p[1]')
        time = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[5]/div[3]/p/a')
        type = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[3]/span')
        starReview = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[4]/span[1]/a')
        blogReview = driver.find_elements(By.XPATH, '//*[@id="info.search.place.list"]/li/div[4]/a/em')


        # 데이터를 담을 리스트 초기화
        placesDF = {
            '이름': [],
            '별점': [],
            '주소': [],
            '영업시간': [],
            '카테고리': [],
            '별점 리뷰수': [],
            '블로그 리뷰수': [],
        }

        # 식당 정보를 리스트에 추가
        for i in range(len(restaurant)):
            placesDF['이름'].append(restaurant[i].text)
            placesDF['별점'].append(star[i].text)
            placesDF['주소'].append(address[i].text)
            placesDF['영업시간'].append(time[i].text)
            placesDF['카테고리'].append(type[i].text)
            placesDF['별점 리뷰수'].append(starReview[i].text)
            placesDF['블로그 리뷰수'].append(blogReview[i].text)

        # DataFrame 생성
        placesDF = pd.DataFrame(placesDF)

        driver.quit()

        return placesDF

In [19]:
get = CurrentLocation()
placesDF = get.restaurant_list()

In [20]:
placesDF

,이름,별점,주소,영업시간,카테고리,별점 리뷰수,블로그 리뷰수
0,엄마마늘보쌈,4.2,서울 노원구 석계로 16,월~토 11:30 ~ 23:00,"족발,보쌈",39건,110
1,무명칼국수,4.3,서울 노원구 석계로7길 4,매일 11:00 ~ 20:00,칼국수,75건,127
2,연어시장,4.6,서울 노원구 석계로 13,월~금 14:00 ~ 23:30,회,51건,36
3,화로상회 광운대점,3.8,서울 노원구 광운로 37 1층,매일 14:00 ~ 00:30,"육류,고기",16건,6
4,오메가생선구이 본점,4.3,서울 노원구 화랑로 343,매일 10:00 ~ 22:00 · 휴게시간 월~금 15:00 ~ 16:30,"해물,생선",20건,17
5,엉터리양평해장국 석계역점,3.8,서울 노원구 석계로1길 14,매일 00:00 ~ 24:00,해장국,6건,0
6,남송통닭,3.2,서울 노원구 석계로3길 30 1층,매일 12:00 ~ 21:00,치킨,5건,6
7,서민초밥,4.0,서울 노원구 석계로3길 17,"첫째,셋째 화요일","초밥,롤",18건,16
8,일심해장국 월계점,3.2,서울 노원구 석계로15길 25 한일아파트상가 1층 108호,매일 00:00 ~ 24:00,해장국,15건,15
9,장군식당,5.0,서울 노원구 화랑로43가길 27 1층,월~토 16:00 ~ 24:00,"족발,보쌈",2건,2
